In [88]:
import time
import math

from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

def get_instruction_tuple(raw):
    o = []
    
    num = ''
    d = ''
    for ch in raw:
        if ch in ['L', 'R']:
            d = ch
            o.append(int(num))
            o.append(d)
            d = ''
            num = ''
        else:
            num += ch
    o.append(int(num))
    return o

def find_other_side(coords, rocks, facing, loc, max_x, max_y):
    x, y = None, None
    if facing in ['l', 'r']:
        y = loc[1]
    else:
        x = loc[0]
    
    if x is not None:
        bound = max_y
        range_ = range(bound+2)
        if facing == 'u':
            range_ = range(bound+10, 0, -1)
        for y in range_:
            if (x, y) in coords:
#                 print('found ud', facing, loc, (x,y))
                return (x, y)
            elif (x, y) in rocks:
                return None
    elif y is not None:
        bound = max_x
        range_ = range(bound+2)
        if facing == 'l':
            range_ = range(bound+10, 0, -1)
        for x in range_:
            if (x, y) in coords:
#                 print('found lr', facing, loc, (x,y))
                return (x, y)
            elif (x, y) in rocks:
                return None
            
    raise Exception(f'not found {facing} {loc}')
            
def visualize(path, dots, walls, x1, y1, x2, y2):
    mapping = {
        'l': '<',
        'r': '>',
        'u': '^',
        'd': '∨',
    }
    coord_to_path_idx = {c[1]: idx for idx, c in enumerate(path)}
    rows = []
    for y in range(y1, y2+1):
        row = ''
        for x in range(x1, x2+1):
            coord = (x, y)
            if coord in coord_to_path_idx:
                idx = coord_to_path_idx[coord]
                
                if idx < (len(path)-1):
                    d = path[idx][0]
                    row += mapping[d]
                else:
                    row += 'X'
            elif coord in walls:
                row += '#'
            elif coord in dots:
                row += '.'
            else:
                row += ' '
                
        rows.append(row)
    return '\n'.join(rows)


def main(path_start=0):      
    with open('../inputs/22.txt') as f:
        y = 1
        start = None
        dots = set()
        rocks = set()
        ins = []
        max_x, max_y = 0, 0
        for line in f:
            if not line.strip():
                raw = next(f)
                ins = get_instruction_tuple(raw)

                break
            for x, ch in enumerate(line):
                x = x + 1
                if ch == '.':
                    dots.add((x, y))
                    max_x, max_y = max(x, max_x), max(y, max_y)
                    if not start:
                        start = (x, y)
                elif ch == '#':
                    rocks.add((x, y))
                    max_x, max_y = max(x, max_x), max(y, max_y)
            y += 1

        f_to_points = {
            'r': 0,
            'd': 1,
            'l': 2,
            'u': 3,
        }

        v = {
            'r': (1, 0),
            'd': (0, 1),
            'l': (-1, 0),
            'u': (0, -1),   
        }

        t = {
            'r': { 'L': 'u', 'R': 'd'},
            'u': { 'L': 'l', 'R': 'r'},
            'd': { 'L': 'r', 'R': 'l'},
            'l': { 'L': 'd', 'R': 'u'}

        }


        loc = start
        facing = 'r'
        path = [(facing, loc)]

        for instruction in ins:
#             print(loc,facing,instruction)
            if instruction in ['L', 'R']:
                facing = t[facing][instruction]
                path[-1] = (facing, path[-1][1])
            else:
                steps = instruction
                for i in range(steps):
                    n = (loc[0]+v[facing][0], loc[1]+v[facing][1])
                    if n in rocks:
                        break
                    elif n in dots:
                        loc = n
                        path.append((facing, loc))
                        continue
                    else:

                        os = find_other_side(dots, rocks, facing, loc, max_x, max_y)
                        if os:

                            loc = os
                            path.append((facing, loc))
                            continue
                        else:
                            break

        

        
#         for i in range(path_start, len(path)):
#             clear_output(wait=True)
#             display(HTML("<style>.container { width:100% !important; }</style>"))

#             print(i, path[i])
#             print(visualize(path[:i+1], dots, rocks, 0, 0, max_x, max_y))
#             time.sleep(.5)
            
        print('answer', max_x, max_y, loc, facing, f_to_points[facing], 1000*(loc[1]) + 4 * (loc[0]) + f_to_points[facing])

In [89]:
main(0)

answer 150 200 (46, 162) l 2 162186


In [3]:
# 162070 too low

In [65]:
x = open("../inputs/22.txt", "r").read()
grid, ins = x.split("\n\n")

### Parsing ###
grid = grid.split("\n")
width = max([len(line) for line in grid])
height = len(grid)
grid = [" " + line.ljust(width) + " " for line in grid]
width += 2
height += 2
grid = [" "*width] + grid + [" "*width]

ins_list = []
buffer = ""
for c in ins.strip("\n"):
    if c in "LR":
        if buffer != "":
            ins_list += [int(buffer)]
            buffer = ""
        ins_list += [c]
    else:
        buffer += c

if buffer != "":
    ins_list += [int(buffer)]
    buffer = ""

### Starting Position ###
x=y=1 #NOTE: pos is 1 index to accomidate buffer for bounds checking
dir = 0
dir_lookup = [[0,1],[1,0],[0,-1],[-1,0]]
for j in range(width):
    if grid[1][j] == ".":
        y=j
        break
dir_cache = (x,y)

### Part 1 ###
for step in ins_list:
    print(x,y,dir,step)
    if step == "R":
        dir = (dir+1)%4
    elif step == "L":
        dir = (dir-1)%4
    else:
        dx,dy = dir_lookup[dir]
        for i in range(step):
            nx,ny = x+dx,y+dy
            if grid[nx][ny] == " ":
                while True:
                    nx,ny = nx-dx,ny-dy
                    if grid[nx][ny] == " ":
                        break
                nx,ny = nx+dx,ny+dy
            if grid[nx][ny] == "#":
                break
            x,y = nx,ny

print('dims', len(grid[0]), len(grid))
print(x,y,dir, x*1000+y*4+dir)

### Face size (work with both input and test data, and other sizes in general) ###
face_size = round(pow(sum([c != " " for line in grid for c in line])//6,0.5))

### Generate initial net adajencies ###
Q = [dir_cache]
visited = {dir_cache:[None]*4}
while len(Q) > 0:
    #print(Q)
    v, *Q = Q
    x,y=v
    for dir in range(4):
        i,j = dir_lookup[dir]
        i,j = x+i*face_size,y+j*face_size
        if not(0 <= i < height and 0 <= j < width):
            continue
        if grid[i][j] == " ":
            continue
        w = (i,j)
        if w not in visited:
            visited[v][dir] = w
            w_list = [None]*4
            w_list[(dir+2)%4] = v
            visited[w] = w_list
            Q += [w]

### Normaize face-edge mapping ###
faces = {}
for i,j in visited:
    faces[(i//face_size,j//face_size)] = [((v[0]//face_size,v[1]//face_size) if v is not None else v) for v in visited[(i,j)]]

### Fill in missing edge data using corners ###
while sum([edge is None for key in faces for edge in faces[key]]) > 0:
    for face in faces:
        for dir in range(4):
            if faces[face][dir] is None:
                for delta in -1,1:
                    common_face = faces[face][(dir+delta)%4]
                    if common_face is None:
                        continue
                    common_face_edge = faces[common_face].index(face)
                    missing_face = faces[common_face][(common_face_edge+delta)%4]
                    if missing_face is None:
                        continue
                    missing_face_edge = faces[missing_face].index(common_face)
                    faces[missing_face][(missing_face_edge+delta)%4] = face
                    faces[face][dir] = missing_face
                    break

# DEBUG
#for key in faces: print(key, faces[key])

### Part 2 ###
x,y = dir_cache
dir = 0
edge_top_offset_out = [[1,1],[1,face_size],[face_size,face_size],[face_size,1]]
for step in ins_list:
    #print("step",x,y,dir,step)
    if step == "R":
        dir = (dir+1)%4
    elif step == "L":
        dir = (dir-1)%4
    else:
        dx,dy = dir_lookup[dir]
        new_dir = dir
        for i in range(step):
            nx,ny = x+dx,y+dy
            if grid[nx][ny] == " ":
                #compute current edge prop
                cur_face = (x-1)//face_size,(y-1)//face_size
                cur_offset = 0
                while tuple([a*face_size+b+c*cur_offset for a,b,c in zip(cur_face,edge_top_offset_out[(dir+1)%4],dir_lookup[(dir+1)%4])]) != (x,y):
                    cur_offset += 1
                #Compute next edge prop
                next_face = faces[cur_face][dir]
                new_dir = (faces[next_face].index(cur_face) + 2) % 4
                nx,ny = tuple([a*face_size+b+c*cur_offset for a,b,c in zip(next_face,edge_top_offset_out[new_dir],dir_lookup[(new_dir+1)%4])])
            if grid[nx][ny] == "#":
                break
            else:
                x,y = nx,ny
                dir = new_dir
                dx,dy = dir_lookup[dir]

print(x*1000+y*4+dir)

1 51 0 36
1 68 0 R
1 68 1 27
1 68 1 L
1 68 0 7
1 68 0 R
1 68 1 50
1 68 1 R
1 68 2 50
1 51 2 L
1 51 1 10
1 51 1 L
1 51 0 1
1 52 0 R
1 52 1 4
5 52 1 R
5 52 2 47
5 138 2 L
5 138 1 15
20 138 1 L
20 138 0 12
20 150 0 R
20 150 1 49
30 150 1 R
30 150 2 18
30 132 2 R
30 132 3 49
17 132 3 R
17 132 0 43
17 140 0 R
17 140 1 8
25 140 1 L
25 140 0 46
25 53 0 R
25 53 1 15
40 53 1 R
40 53 2 21
40 135 2 L
40 135 1 15
41 135 1 R
41 135 2 24
41 118 2 L
41 118 1 48
6 118 1 L
6 118 0 13
6 126 0 L
6 126 3 44
45 126 3 R
45 126 0 35
45 133 0 L
45 133 3 33
29 133 3 L
29 133 2 34
29 127 2 L
29 127 1 22
33 127 1 R
33 127 2 31
33 117 2 L
33 117 1 33
33 117 1 L
33 117 0 38
33 133 0 L
33 133 3 15
29 133 3 L
29 133 2 1
29 132 2 R
29 132 3 43
17 132 3 R
17 132 0 49
17 140 0 R
17 140 1 46
26 140 1 L
26 140 0 36
26 144 0 R
26 144 1 3
29 144 1 L
29 144 0 38
29 146 0 L
29 146 3 35
24 146 3 L
24 146 2 50
24 144 2 L
24 144 1 8
32 144 1 L
32 144 0 6
32 144 0 L
32 144 3 38
19 144 3 L
19 144 2 11
19 133 2 R
19 133 3 15
4 133